## Compare OSM with IMD 2018

**Inputs:** 

* Path to OSM File;

* Path to IMD Raster;

* Table with relation between OSM tags and LULC Classes

* Path to output.

In [ ]:
osmfile = '/home/jasp/mystuff/dgt/osm_parts/osmpt_74.xml'

imdrst  = '/home/jasp/mystuff/dgt/imd2018/imd_74_v2.tif'

osm_ref_tags = {
    "TABLE"     : '/home/jasp/mystuff/dgt/osm_features_2021_v2.xlsx',
    "SHEET"     : 'osm_features',
    "LULC_COL"  : 'L4',
    "KEY_COL"   : "key",
    "VALUE_COL" : "value",
    "GEOM_COL"  : "geom"
}

outgrid = '/home/jasp/mystuff/dgt/osmvsimd/osmimd_74.shp'

Constants

In [ ]:
osmgeomtable = "multipolygons"

classes_edificado = [
    '1141', '1151', '1211', '1221',
    '1222', '1223', '1231', '1241',
    '1251', '1252', '1254', '1255',
    '1256', '1257', '1253', '1612',
    '1631', '1632', '1633', '1651',
    '16', '143', '1431', '1432'
]

Import modules

In [ ]:
import os
import pandas as pd
import numpy as np

from glass.it.osm import osm_to_psql
from glass.it.shp import dbtbl_to_shp
from glass.prop.sql import cols_name
from glass.rd import tbl_to_obj
from glass.sql.q import exec_write_q
from glass.sql.db import create_db

from glass.pys.oss import mkdir, fprop
from glass.smp.fish import nfishnet_fm_rst
from glass.prop.prj import get_epsg

Create folder to save temporary files

In [ ]:
tmp = mkdir(os.path.join(os.path.dirname(outgrid), 'tmpdata'), overwrite=True)

Get IMD EPSG code

In [ ]:
epsg = get_epsg(imdrst)

Generate Fishnet based on imdrst

In [ ]:
fishfolder = mkdir(os.path.join(tmp, 'fishnet'))

fishnets = nfishnet_fm_rst(imdrst, 500, 500, fishfolder)

Get LULC from OSM

In [ ]:
# Import OSM data into a database
osmdb = create_db(f'osm_{fprop(outgrid, "fn")}', api='psql', overwrite=True)

osm_to_psql(osmfile, osmdb)

# Read table with OSM Tags and related LULC Classes
osm_tags = tbl_to_obj(osm_ref_tags["TABLE"], sheet=osm_ref_tags["SHEET"])

osm_tags = osm_tags[osm_tags[osm_ref_tags["GEOM_COL"]] == 'Polygon']

# Remove values with ';' in LULC_COL
osm_tags['sevtags'] = osm_tags[osm_ref_tags["LULC_COL"]].str.contains(';')

osm_tags = osm_tags[osm_tags.sevtags != True]

# Create key/value column
osm_tags.loc[:, osm_ref_tags["VALUE_COL"]] = osmgeomtable + "." + \
    osm_tags[osm_ref_tags["KEY_COL"]] + \
    "='" + osm_tags[osm_ref_tags["VALUE_COL"]] + "'"

# Add new column to multipolygons table

cols = cols_name(osmdb, osmgeomtable)

qs = [] if "lulc_cls" in cols else [(
    f"ALTER TABLE {osmgeomtable} ADD COLUMN "
    "lulc_cls integer"
)]

# Create Queries to update multipolygons table
# Queries to add lulc classe
for cls in osm_tags[osm_ref_tags["LULC_COL"]].unique():
    # Se uma feature estiver associada a duas TAGS que dizem respeito a classes
    # diferentes, a classe da feature será a última classe considerada
    # Abordagem multitag vai resolver este problema.
    __osmtags = osm_tags[osm_tags[osm_ref_tags["LULC_COL"]] == cls]
    
    qs.append((
        f"UPDATE {osmgeomtable} SET lulc_cls={str(cls)} "
        f"WHERE {str(__osmtags[osm_ref_tags['VALUE_COL']].str.cat(sep=' OR '))}"
    ))

# Run queries
exec_write_q(osmdb, qs, api='psql')

# Export shapefile with data
whr = " OR ".join([f"lulc_cls={c}" for c in classes_edificado])

q = (
    f'SELECT ogc_fid, osm_id, name, lulc_cls, '
    'building, amenity, landuse, '
    f'ST_Transform(wkb_geometry, {str(epsg)}) AS geom '
    f'FROM {osmgeomtable} '
    f'WHERE {whr}'
)

osmlulc = dbtbl_to_shp(
    osmdb, q, 'geom', os.path.join(tmp, 'osmlulc.shp'),
    tableIsQuery=True, api='pgsql2shp', epsg=epsg
)